In [2]:
import numpy as np
import tensorflow as tf
import random, math
import matlab
import matlab.engine as me
from dataSetup import generateData, generateWeights
from recovAnalysis import recovery, structDiff
from trainerUtil import tensorInit, train

In [ ]:
### Implementation Verification

# m[2,3]/np.linalg.norm(W_gt[:,3])**2

# M =  np.asarray(getM2(train_x, train_y))

# check = np.zeros((d,d))
# # check = np.zeros(d)
# for i in range(k):
#     check += v_gt[i]*m[1,i]*np.outer(W_gt[:,i], W_gt[:,i])
# #     check += v_gt[i]*m[0,i]*W_gt[:,i]
# print check
# print np.linalg.norm(M-check)/np.linalg.norm(check)
# print np.linalg.norm(M-check)/np.linalg.norm(M)

In [3]:
n = 2000
test_n = 1000
d = 10
k = 5
thresh_gt = 0.15
thresh_train = 0.15
batch_size = 20
lr = 1e-3
noise_sd = 0.1

w_gt, v_gt, m =  generateWeights(d, k, thresh_gt)
train_x, train_y, test_x, test_y = generateData(w_gt, v_gt, n, test_n, d)
train_y_noisy = train_y + np.random.normal(0, noise_sd, n)

eng = me.start_matlab()
tensorWeights = tensorInit(train_x, train_y_noisy, w_gt, m ,k, eng)
eng.quit()

w_res, train_loss, test_loss = train(train_x, train_y_noisy, test_x, test_y, tensorWeights,v_gt, thresh_train)   

recoveryVal = recovery(w_gt, v_gt, w_res, v_gt)
print recoveryVal

recoveryStructure = structDiff(w_gt, w_res, 1e-2)
print recoveryStructure

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
NT = 10

y_tl0 = y_tl0[:NT]
acc0 = acc0[:NT]

y_tl1 = y_tl1[:NT]
acc1 = acc1[:NT]

y_tl2 = y_tl2[:NT]
acc2 = acc2[:NT]

y_tl3 = y_tl3[:NT]
acc3 = acc3[:NT]

# y_tl4 = y_tl4[:NT]
# acc4 = acc4[:NT]
x_e = x_e[:NT]

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(a, b, 'blue', label="Ground Truth Thresh= 0.20")
ax1.set_xlabel('threshold', color="white")
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('recovery', color='white')
ax1.tick_params('y', colors='white')
ax1.tick_params('x', colors='white')
l1 = ax1.legend(bbox_to_anchor=(1.1 ,0.5), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(x_e, y_tl0, 'blue', label="Ground Truth Thresh= 0.15")
# ax1.plot(x_e, y_tl, 'red', label="random")
ax1.plot(x_e, y_tl1, 'red', label="thresh = 0.10")
ax1.plot(x_e, y_tl2, 'green', label="thresh=0.15")
ax1.plot(x_e, y_tl3, 'black', label="thresh=0.20")
ax1.plot(x_e, y_tl4, 'magenta', label="thresh=0.25")
ax1.set_xlabel('epochs', color="white")
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('training loss', color='white')
ax1.tick_params('y', colors='white')
ax1.tick_params('x', colors='white')
l1 = ax1.legend(bbox_to_anchor=(1.1 ,0.5), loc=2, borderaxespad=0.)
# ax1.set_ylim([0.20,0.30])


# ax2 = ax1.twinx()
fig, ax2 = plt.subplots()
ax2.plot(x_e, acc0, 'blue', label="Ground Truth Thresh= 0.15")
# ax2.plot(x_e, acc, 'red', label="random_acc")
ax2.plot(x_e, acc1, 'red',label="thresh=0.10")
ax2.plot(x_e, acc2, 'green',label="thresh=0.15")
ax2.plot(x_e, acc3, 'black',label="thresh=0.20")
ax2.plot(x_e, acc4, 'magenta',label="thresh=0.25")
ax2.set_ylabel('test error', color='white')
ax2.tick_params('y', colors='white')
ax2.tick_params('x', colors='white')
ax2.set_xlabel('epochs', color="white")
l2 = ax2.legend(bbox_to_anchor=(1.1, 1), loc=2, borderaxespad=0.)
# ax2.set_ylim([0.20,0.30])
# 
# fig.tight_layout()
plt.show()